# Extracción de requisitos no funcionales
En este notebook utilizamos los modelos exportados para poder clasificar una frase o un conjunto de frases. Los modelos que se utilizarán en este notebook serán los que se encuentren en el directorio **/models** del repositorio **nfr-extraction**. 

Por defecto estaremos utilizando SVM, aunque si ya hemos ejecutado el notebook **results.ipynb**, habremos sobreescrito los modelos del directorio **/models** y estaremos utilizando los que hayamos indicado en esa ejecución.

### Activamos drive

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importamos librerías necesarias

In [ ]:
import re
import io
import os
import ipywidgets as widgets
from joblib import load

import nltk
from nltk.corpus import wordnet
from sklearn.preprocessing import normalize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

!pip install spacy
!python -m spacy download en_core_web_lg

import spacy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


### Definimos función que nos dice si una palabra es un adjetivo, un nombre, un verbo o un advervio.

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

### Definimos función que recibe una cadena de texto y devuelve el texto procesado.

In [ ]:
def preprocess(sentence):
    # Convertimos a minúsculas
    new_text = sentence.lower()
    
    # Eliminamos puntuación
    new_text = re.sub(r'[^\w\s]', '', new_text)

    # Dividimos en tokens
    tokens = nltk.tokenize.TreebankWordTokenizer().tokenize(new_text)

    # Eliminamos stopwords
    tokens = [word for word in tokens if not word in nltk.corpus.stopwords.words('english')]

    # Stemming
    # stemmer = nltk.stem.PorterStemmer()
    # new_text = ' '.join([stemmer.stem(w) for w in tokens])
    
    # lemma
    lemmatizer = nltk.stem.WordNetLemmatizer()
    new_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens])

    # new_text = ' '.join([w for w in tokens])

    # Reemplazamos números por #s
    if bool(re.search(r'\d', new_text)):
        new_text = re.sub('[0-9]{5,}', '#####', new_text)
        new_text = re.sub('[0-9]{4}', '####', new_text)
        new_text = re.sub('[0-9]{3}', '###', new_text)
        new_text = re.sub('[0-9]{2}', '##', new_text)
        # Cuando existe un solo número lo eliminamos
        new_text = re.sub('[0-9]{1}', '', new_text)

    return new_text

### Modos de ejecución 
Para utilizar el clasificador tenemos dos modos de ejecución:
1. Clasificar frase individual.
2. Clasificar conjunto de frases.

Para ello sólo tenemos que seleccionar en el siguiente desplegable el modo de ejecución que queremos. 


In [ ]:
mode_list = [('Conjunto de frases', 0), ('Frase individual', 1)]
mode_picker = widgets.Dropdown(options=mode_list)
print("Selecciona un modo de ejecución: ")
mode_picker

Selecciona un modo de ejecución: 


Dropdown(options=(('Conjunto de frases', 0), ('Frase individual', 1)), value=0)

En este punto debemos, o introducir una frase o subir un archivo **.txt** con un conjunto de frases, según la opción elegida en la celda anterior. Para la lectura correcta del archivo las frases deben ir escritas en líneas distintas.

In [ ]:
if mode_picker.value == 1:
    sentences = input('Introduce una frase para extraer sus requisitos no funcionales: ')
    sentences = [sentences]
else:
    uploaded = files.upload()
    sentences_file = open(list(uploaded.keys())[-1], 'r') 
    sentences = sentences_file.readlines()
    sentences = [sentence.strip() for sentence in sentences]

Saving sentences.txt to sentences.txt


### Cargamos el modelo GloVe para poder realizar las conversiones de cadena de texto a vector de números

In [ ]:
nlp = spacy.load('en_core_web_lg')

### Indicamos categorías
Aquí definimos las categorías existentes así como las traducciones de estas al español. Como podemos ver no hemos incluido la categoría *not applicable*. Esto es debido a que esta categoría sólo se asignarán a aquellas frases que no tengan otras categorías.

In [ ]:
# Indicamos las categorías existentes, excepto notapplicable
categories = [
    "access control", "audit", "availability", "legal", "look and feel",
    "maintainability", "operational", "privacy", "recoverability", "capacity and performance",
    "reliability", "security", "usability", "other nonfunctional", "functional"
]
# Creamos diccionario con las traducciones para cada categoría
cat_translations = {
    "access control": "Control de acceso", 
    "audit": "Auditoría",
    "availability": "Disponibilidad",
    "legal": "Legal",
    "look and feel": "Diseño",
    "maintainability": "Mantenibilidad",
    "operational": "Operacional",
    "privacy": "Privacidad",
    "recoverability": "Recuperabilidad",
    "capacity and performance": "Rendimiento",
    "reliability": "Fiabilidad",
    "security": "Seguridad",
    "usability": "Usabilidad",
    "other nonfunctional": "Otros no funcionales",
    "functional": "Funcional",
    "not applicable": "No aplicable"
}

###Realizamos el proceso de clasificación

In [ ]:

processed_sentences = [preprocess(sentence) for sentence in sentences]

x_test = normalize([nlp(s).vector for s in processed_sentences])

for i, sentence in enumerate(processed_sentences):
    requirements = []
    # Pasamos la frase por cada clasificador (1 por categoría)
    for category in categories:
        model = load('./drive/My Drive/nfr-extraction/models-glove/' + category.replace(" ", "") + ".joblib")
        requirements.extend(model.predict([x_test[i]]))

    prediction = [category for indx, category in enumerate(categories) if requirements[indx]]
    
    if prediction == []:
        prediction = ['not applicable']

    # Mostramos categorías identificadas
    print("\n\n" + sentences[i])
    print("Categorías identificadas: ", end="")
    for predict in prediction:
        print(cat_translations[predict], end=", ")
    
if mode_picker.value == 0:
    try:
        os.remove(list(uploaded.keys())[-1])
    except:
        pass



security entity
Categorías identificadas: No aplicable, 

talking about papers and connection
Categorías identificadas: Funcional, 

And remember that OpenEMR's username and passwords are case sensitive.
Categorías identificadas: Seguridad, 

A hashing algorithm with a security strength equal to or greater than SHA-1 (Secure Hash Algorithm (SHA-1) as specified by the National Institute of Standards and Technology (NIST) in FIPS PUB 180-4 (March 2012)) must be used to verify that electronic health information has not been altered.
Categorías identificadas: Mantenibilidad, Seguridad, 

The date, time, patient identification, user identification, and a description of the disclosure must be recorded for disclosures for treatment, payment, and health care operations, as these terms are defined at 45 CFR 164.501.
Categorías identificadas: Privacidad, 

A covered entity may, consistent with applicable law and standards of ethical conduct, use or disclose protected health information, if the